In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord

In [2]:
# Data for NGC188
data = fits.getdata("NGC188.fits.gz")

FileNotFoundError: [Errno 2] No such file or directory: 'NGC188.fits.gz'

In [ ]:
'011.798	+85.244 	0.272 	857 	-2.307 	-0.960 	0.507	NN 	9.85 	0.21	11.15 	1698 	-851 	1319 	646 	9285	'.split('\t')

In [ ]:
# Parameters for NGC188: https://dx.doi.org/10.1051/0004-6361/201833476, http://dx.doi.org/10.1051/0004-6361/202038192
r50 = 14 * u.arcmin
ra_c = 11.798
dec_c = 85.244
dist_kpc = 1698/1000

In [ ]:
plt.style.use("report.mplstyle")

In [ ]:
# Astrometry
ra = data['ra']
dec = data['dec']

pmra = data['pmra']
pmdec = data['pmdec']

parallax = data['parallax']
e_parallax = data['parallax_error']

In [ ]:
# Photometry
g = data['phot_g_mean_mag']
g_rp = data['g_rp']

In [ ]:
# Systematics
ruwe = data['ruwe']
aen = data['astrometric_excess_noise']
aen_sigma = data['astrometric_excess_noise_sig']

In [ ]:
g_flux = data['phot_g_mean_flux']
size = g_flux/np.nanmax(g_flux) * 100

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.scatter(np.remainder(ra-180, 360), dec, s=size, alpha=0.7)
plt.xticks(plt.xticks()[0], np.remainder(plt.xticks()[0]+180, 360))

plt.subplot(122)
plt.scatter(pmra, pmdec, s=size, alpha=0.7)

In [ ]:
plt.hist2d(pmra, pmdec, bins=np.linspace(-20, 20, 100), vmin=0, vmax=300);
plt.xlabel(r"$\mu_\alpha\ \cos \delta$")
plt.ylabel(r"$\mu_\delta$")
# plt.colorbar()

In [ ]:
sel_plx = (parallax - 1/dist_kpc) > 3 * e_parallax
sel_ruwe = ruwe > 1.4
sel_aen = (aen>1)
sel_pm = np.isnan(pmra) | np.isnan(pmdec)
sel_nan = np.isnan(g) | np.isnan(g_rp)

In [ ]:
sel_good_astrometry = ~(sel_plx | sel_ruwe | sel_aen | sel_pm | sel_nan)

In [ ]:
obs_coords = SkyCoord(ra, dec, unit='deg')
cluster_coords = SkyCoord(ra_c, dec_c, unit='deg')

## Generate training set for PRF

In [ ]:
sel_r50 = (cluster_coords.separation(obs_coords) < r50)
sel_train = sel_good_astrometry & sel_r50

In [ ]:
mu = np.c_[pmra, pmdec][sel_train]

In [ ]:
def univariate_gaussian(x, A, mu, sig):
    """Un-normalized Gaussian"""
    return A * np.exp(-(x - mu)**2 / (2 * sig**2))


def multivariate_gaussian(X, mu, Sigma):
    """Multivariate Gaussian distribution"""
    k = mu.shape[0]

    V = np.linalg.inv(Sigma)
    norm = np.sqrt((2 * np.pi)**k * np.linalg.det(Sigma))

    res = X - mu

    return 1 / norm * np.exp(-1 / 2 * np.einsum('...j,jk,...k', res, V, res))

In [ ]:
# Cluster
w_c = 0.5
mu_c = np.array([-3,-2])
Sigma_c = np.array([[0.1, 0], [0, 0.1]])

# Field
w_f = 1 - w_c
mu_f = np.array([0, 0])
Sigma_f = np.array([[400, 0], [0, 400]])

In [ ]:
for i in range(100):
    pm_dist_c = w_c * multivariate_gaussian(mu, mu_c, Sigma_c)
    pm_dist_f = w_f * multivariate_gaussian(mu, mu_f, Sigma_f) 
    pm_dist = pm_dist_c + pm_dist_f

    member_probability = pm_dist_c/pm_dist
    field_probability = pm_dist_f/pm_dist

    w_c = np.sum(member_probability)/mu.shape[0]
    mu_c = np.average(mu, weights=member_probability, axis=0)
    Sigma_c = np.einsum('ji,ik->jk', member_probability*(mu-mu_c).T, mu-mu_c)/member_probability.sum()

    w_f = np.sum(field_probability)/mu.shape[0]
    mu_f = np.average(mu, weights=field_probability, axis=0)
    Sigma_f = np.einsum('ji,ik->jk', field_probability*(mu-mu_f).T, mu-mu_f)/field_probability.sum()

In [ ]:
mu_c

In [ ]:
mu_f

In [ ]:
plt.hist(member_probability, bins=20);

In [ ]:
member_cluster = member_probability > 0.5

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.scatter(ra[sel_train],
            dec[sel_train],
            s=size[sel_train] * 10)
plt.scatter(ra[sel_train][member_cluster],
            dec[sel_train][member_cluster],
            s=size[sel_train][member_cluster] * 10)

plt.subplot(122)
plt.scatter(pmra[sel_train],
            pmdec[sel_train],
            s=size[sel_train] * 10)
plt.scatter(pmra[sel_train][member_cluster],
            pmdec[sel_train][member_cluster],
            s=size[sel_train][member_cluster] * 10)
plt.xlim(-20,20)
plt.ylim(-20,20)

In [ ]:
plt.scatter(g_rp[sel_train][member_cluster], g[sel_train][member_cluster], s=1)
plt.gca().invert_yaxis()

In [ ]:
training_data = data[sel_train]

In [ ]:
from astropy.table import Table

In [ ]:
tab = Table(training_data)


In [ ]:
tab.add_column(member_cluster.astype("int"), name='Class')
tab.add_column(member_probability, name='mem_prob')
pmr0 = np.linalg.norm(mu-mu_c, axis=1)
tab.add_column(pmr0, name='pmrzero')

In [ ]:
tab.write("Train_NGC188_csv.dat", format='csv', overwrite=True)

## PRF training


In [ ]:
import pandas as pd
data= pd.read_csv("Train_NGC188_csv.dat")
# data.head()

##filtering quality factor
def qf(row):
    if row['ruwe'] > 1.4 or (row['astrometric_excess_noise'] > 1 and row['astrometric_excess_noise_sig']>2):
        return 0
    else:
        return 1

data['qf'] = data.apply(qf,axis=1)

cols = data.columns
#shuffuling the data
# new_data = data.sample(frac=1)

print(cols)

F_10 = ['ra','dec','parallax','pmra','pmdec','pmrzero','g_rp','phot_g_mean_mag','ruwe','astrometric_excess_noise']
F_8 = ['ra','dec','parallax','pmra','pmdec','pmrzero','g_rp','phot_g_mean_mag']
F_6 = ['ra','dec','parallax','pmra','pmdec','pmrzero']
# # F_10 = ['ra']
# print(len(F_10))
data_cols = F_10 + ['Class','mem_prob', 'source_id', 'qf']
data = data[data_cols]
data.head()


In [ ]:
def get_train_test_data(data,frac=3/4, feature=F_10):
    FRAC = frac
    data.dropna()
    split = int(len(data)*FRAC) -1
    
    train_data = data.iloc[:split]
    test_data = data.iloc[split:]


    X_train = train_data[feature]
    y_train = train_data['Class']
    dy_train = train_data['mem_prob']
    dy_train = np.array([(1-i,i) for i in dy_train])

    X_test = test_data[feature]
    y_test = test_data['Class']
    X_train = X_train.values
    X_test = X_test.values
    return X_train,y_train,dy_train, X_test,y_test

In [ ]:
import warnings; warnings.simplefilter('ignore')

import PRF 
from tqdm import tqdm

def prf_res(n_trees=10, keep_prob=0.05):
    prf_cls = PRF.prf(n_estimators=n_trees,  bootstrap=True, keep_proba=keep_prob)
    prf_cls.fit(X=X_train, y=y_train)
    a = prf_cls.score(X=X_test, y=y_test)
    return a

def prf_pred(data, frac=3/4, features={'F10':F_10,'F8':F_8,'F6':F_6}, n_trees=180, keep_prob=0.05, dy_check=False):
    probs = pd.DataFrame()
    scores = {}
    for key,feature in features.items():
        X_train,y_train,dy_train, X_test,y_test = get_train_test_data(data, frac, feature)
        prf_cls = PRF.prf(n_estimators=n_trees,  bootstrap=True, keep_proba=keep_prob)
        if dy_check:
            prf_cls.fit(X=X_train, py=dy_train)
        else:
            prf_cls.fit(X=X_train, y=y_train)
        score = prf_cls.score(X=X_test, y=y_test)
        prob_ = prf_cls.predict_proba(X=np.array(data))
        prob_ = [j for i,j in prob_]
        probs[key] = prob_
        scores[key] = score
    
    data = pd.concat([data,probs],axis=1)
    
    return data, scores
        
data_,score = prf_pred(data)
print(data_)
print(score) 
    


In [ ]:
ORDER = ['source_id','ra','dec','phot_g_mean_mag','g_rp','qf','F6','F8','F10','mem_prob','Class']
def write_data(data, file_name='NGC188_PRF.dat', order=ORDER ,format='csv'):
    data = data[ORDER]
    data.to_csv(file_name,index=False)
    
def prf_tune():
    prf_results = []

    for i in tqdm(range(10,300,10)):
        prf_tr = []
        for j in range(1,100,10):
            score = prf_res(n_trees=i, keep_prob=j/1000)
            data_, scores = prf_pred(data,n_trees=i, keep_prob=j/1000)
            prf_tr.append([i,j/1000, scores])
        prf_results.append(prf_tr)
    return prf_results





In [ ]:
write_data(data_)
